In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.optimizers import SGD
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import TensorBoard
import tensorflow as tf

os.chdir('../Utils/')
import featureGenerator
from featureGenerator import *
os.chdir('../src/')
import orderbook_lstm
from orderbook_lstm import OrderBookLSTM

Using TensorFlow backend.


# Generate Features and Response Vars

In [2]:
data_dir = '../../../../ProjectData/'
in_path = data_dir+'msft-orderbook.csv'
out_path = data_dir+'msft-orderbook-all.csv'
out_path2 = data_dir+'msft-orderbook-final.csv'
response_type = "Regression"

In [3]:
mergeOrderBookDays(data_dir, out_path, ['msft'])

['msft-20170417-orderbook.csv', 'msft-20170418-orderbook.csv', 'msft-20170419-orderbook.csv', 'msft-20170420-orderbook.csv', 'msft-20170421-orderbook.csv', 'msft-20170424-orderbook.csv', 'msft-20170425-orderbook.csv', 'msft-20170426-orderbook.csv', 'msft-20170427-orderbook.csv', 'msft-20170428-orderbook.csv']
../../../../ProjectData/msft-20170417-orderbook.csv
../../../../ProjectData/msft-20170418-orderbook.csv
../../../../ProjectData/msft-20170419-orderbook.csv
../../../../ProjectData/msft-20170420-orderbook.csv
../../../../ProjectData/msft-20170421-orderbook.csv
../../../../ProjectData/msft-20170424-orderbook.csv
../../../../ProjectData/msft-20170425-orderbook.csv
../../../../ProjectData/msft-20170426-orderbook.csv
../../../../ProjectData/msft-20170427-orderbook.csv
../../../../ProjectData/msft-20170428-orderbook.csv


In [4]:
createFeatures(out_path, out_path2, response_type = response_type, look_forward =1)
data = pd.read_csv(out_path2)
data_orig = pd.read_csv(out_path2)

/Users/adam/Desktop/Stanford/MSE 448/448Project/448Project/LSTM/Utils/featureGenerator.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['Response'] = response_col


In [5]:
pd.set_option('display.max_columns', 500)

data.head()

,direct.vwap,meanBid,meanAsk,meanBidNum,meanAskNum,meanBidVol,meanAskVol,accumulatedPriceDiff,accumulatedVolumeDiff,spread_1,spread_2,spread_3,spread_4,spread_5,spread_6,spread_7,spread_8,spread_9,spread_10,midPrice_1,midPrice_2,midPrice_3,midPrice_4,midPrice_5,midPrice_6,midPrice_7,midPrice_8,midPrice_9,midPrice_10,Response
0,65.100361,64.924,65.288,2.7,1.7,2921.6,997.0,3.64,-19246,0.14,0.19,0.24,0.26,0.35,0.41,0.46,0.48,0.54,0.57,65.100,65.095,65.080,65.080,65.095,65.105,65.120,65.120,65.130,65.135,65.081990
1,65.081990,64.970,65.208,6.2,2.8,2439.2,1263.6,2.38,-11756,0.08,0.13,0.15,0.18,0.22,0.24,0.27,0.31,0.35,0.45,65.060,65.075,65.075,65.080,65.090,65.090,65.095,65.105,65.095,65.125,65.097505
2,65.097505,65.044,65.185,5.6,2.8,4393.5,790.1,1.41,-36034,0.04,0.06,0.08,0.11,0.13,0.15,0.18,0.20,0.22,0.24,65.120,65.120,65.120,65.115,65.115,65.115,65.110,65.110,65.110,65.110,65.099488
3,65.099488,65.048,65.185,5.4,2.6,4300.8,319.6,1.37,-39812,0.04,0.06,0.08,0.11,0.13,0.15,0.17,0.19,0.21,0.23,65.120,65.120,65.120,65.115,65.115,65.115,65.115,65.115,65.115,65.115,65.101385
4,65.101385,65.070,65.175,2.7,3.0,2587.2,482.9,1.05,-21043,0.01,0.03,0.05,0.07,0.09,0.12,0.14,0.16,0.18,0.20,65.125,65.125,65.125,65.125,65.125,65.120,65.120,65.120,65.120,65.120,65.101807


In [6]:
data = pd.read_csv(out_path2)
#data = data.drop(['datetime', 'direct.last_SRO'], axis = 1)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
cols_to_normalize = [col for col in data.columns if col != 'Response']
data[cols_to_normalize] = scaler.fit_transform(data[cols_to_normalize])

dataset = data.values
dataset = dataset.astype('float32')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(20, 20))
corr = data.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)
plt.show()

In [ ]:
# Plots
data_orig = pd.read_csv(out_path)

plt.plot(np.arange(len(data_orig)), data_orig['direct.bsize1'])
plt.plot(np.arange(len(data_orig)),data_orig['direct.bsize2'])
plt.plot(np.arange(len(data_orig)),data_orig['direct.bsize3'])
plt.plot(np.arange(len(data_orig)),data_orig['direct.bsize4'])
plt.plot(np.arange(len(data_orig)),data_orig['direct.bsize5'])


In [ ]:
plt.show()

# Train/Test Split

In [7]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

156779 77220


In [19]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back, response_type):
    if response_type == 'Classification':
        dataY = get_one_hot(dataset[look_back+1:,dataset.shape[1]-1].astype(int).reshape(-1),3)
        dataX = []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), :]    
            dataX.append(a)
        return np.array(dataX), np.array(dataY)
    
    elif response_type == 'Regression':
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), :]
            dataX.append(a)
            dataY.append(dataset[i, dataset.shape[1]-1])
            #print(i + look_back)
        return np.array(dataX), np.array(dataY)
        

In [20]:
# Convert response variable to one-hot vectors
def get_one_hot(targets, nb_classes):
    return np.eye(nb_classes)[np.array(targets).reshape(-1)]

In [21]:
look_back = 30
trainX, trainY = create_dataset(train, look_back, response_type = response_type)
testX, testY = create_dataset(test, look_back, response_type = response_type)


In [30]:
trainX[1].shape

(30, 30)

# Train Model

In [31]:
from importlib import reload
os.chdir('../src/')
import orderbook_lstm
from orderbook_lstm import OrderBookLSTM
reload(orderbook_lstm)

#tf.reset_default_graph()
from keras import backend as K
K.clear_session()
#timesteps = 10
n_features = 69
n_neurons = 100
n_classes = 3
n_hidden = 1
dropout = None

#lstm = OrderBookLSTM(lookback, n_neurons, (timesteps,n_features), n_classes, n_hidden, response_type, dropout)

lstm = OrderBookLSTM(timesteps = look_back, 
                     layer_neurons = 100, 
                     input_shape = (look_back,trainX.shape[2]), 
                     output_shape = 3, # doesn't matter if regression
                     num_hidden_layers = 0, 
                     response_type = response_type)



Building regression model...
Compiling model...


In [32]:
mod = lstm.get_model()

In [33]:
mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 100)           52400     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3001      
Total params: 55,401
Trainable params: 55,401
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Class weights to change
class_weight = {0 : 1.,
    1: 1.,
    2: 1.} 

if response_type == 'Classification':
    mod.fit(trainX, trainY, validation_data=([testX, testY]), 
              epochs=5,  
              batch_size=128, 
              verbose=1, 
              class_weight = class_weight,
              callbacks=[TensorBoard(log_dir='Logs/', write_graph=True)])
    

if response_type == 'Regression':
    mod.fit(trainX, trainY, validation_data=([testX, testY]),
              epochs=5,  
              batch_size=256, 
              verbose=1, 
              callbacks=[TensorBoard(log_dir='Logs/', write_graph=True)])

Train on 156748 samples, validate on 77189 samples
Epoch 1/5
156748/156748 [==============================] - 68s 433us/step - loss: 133.1387 - val_loss: 3.8153
Epoch 2/5
156748/156748 [==============================] - 69s 442us/step - loss: 0.4119 - val_loss: 0.0306
Epoch 3/5
156748/156748 [==============================] - 70s 444us/step - loss: 0.0088 - val_loss: 0.0232
Epoch 4/5
156748/156748 [==============================] - 70s 449us/step - loss: 0.0079 - val_loss: 0.0201
Epoch 5/5
156748/156748 [==============================] - 72s 459us/step - loss: 0.0062 - val_loss: 0.0259


In [ ]:
unique, counts = np.unique(dataset[:,68], return_counts=True)
counts

# Make Predictions and get Metrics

In [ ]:
# Training Error
if response_type == 'Classification':
    preds_training = mod.predict(trainX).argmax(axis=-1)
    pd.Series(preds_training).value_counts()
    precision_recall_fscore_support(np.argmax(trainY, axis=1), preds_training)
else:
    preds_training = mod.predict(trainX)

In [ ]:
# Validation Error
if response_type == 'Classification':
    preds = mod.predict(testX).argmax(axis=-1)
    pd.Series(preds).value_counts()
    print(precision_recall_fscore_support(np.argmax(testY, axis=1), preds))
    preds_probs = mod.predict(testX)
else:
    preds_test = mod.predict(testX)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt

cm = confusion_matrix(class_labels, preds)
df_cm = pd.DataFrame(cm, range(3),
                  range(3))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 10})# font size

In [ ]:
plt.show()

In [ ]:
from collections import Counter
l = list(preds)
Counter(l)

In [ ]:
len(preds)

# Calculate PnL (+1)

In [ ]:
starting_money = 10000
pnl_lstm = 0
buffer = 0.05
pnl_history_lstm = []
pnl_history_by_trade = []

if response_type == 'Regression':
    for i in range(len(testX)):
        pred = preds_test[i][0]
        current = testY[i] 
        
        '''
        At time t:
         - if pred > current:
             - buy at ask price
         
         - if pred < current:
             - sell at bid price
             
             
        At time t+1:
        Liquidate assets
        
         - if bought at time t:
             - sell at bid price at t + 1
             
         - if sold at time t:
             - buy at bid price at t + 1
             
        '''
        buy_bool = None
        buy_price = None
        sell_price = None
        amount_buy = 1
        amount_sell = 1
        
        
        # buy
        if pred > (current + buffer):
            '''
            - Buy at ask price
            - 
            '''
            buy_bool = True
            buy_price = data_orig.loc[i, 'direct.ask1']
            amount_buy = 1000
        # sell
        elif pred < (current - buffer):
            '''
            Sell at bid price
            '''
            buy_bool = False
            sell_price = data_orig.loc[i, 'direct.bid1']
            amount_sell = 1000
        else:
            continue
            
        
        # liquidate
        if buy_bool == True:
            ask_next = data_orig.loc[i+1, 'direct.ask1']
            pnl_lstm += (ask_next - buy_price) * amount_buy
            pnl_history_by_trade.append((ask_next - buy_price) * amount_buy)
            
        else:
            bid_next = data_orig.loc[i+1, 'direct.bid1']
            pnl_lstm += (bid_next - sell_price) * amount_sell
            pnl_history_by_trade.append((bid_next - sell_price) * amount_sell)
        
        pnl_history_lstm.append(pnl_lstm) 
    

# VWAP Crossing

In [ ]:
starting_money = 10000
pnl_lstm = 0
buffer = 0.05
pnl_history_lstm = []
pnl_history_by_trade = []

inventory = 1000
inventory_history = [1000]

test_data = data_orig.loc[train_size:len(data_orig),:].reset_index(drop = True)


n = len(testX)
n = 50
if response_type == 'Regression':
    for i in range(n):
        pred = preds_test[i][0]
        current_ask = test_data.loc[i, 'direct.ask1']
        current_bid = test_data.loc[i, 'direct.bid1']

        '''
        At time t:
         - if pred > current_ask:
             - buy at ask price
         
         - if pred < current_bid:
             - sell at bid price
             
             
        At time t+1:
        Liquidate assets
        
         - if bought at time t:
             - sell at bid price at t + 1
             
         - if sold at time t:
             - buy at bid price at t + 1
             
        '''
        
        
        # buy
        if pred > (current_ask + buffer):
            '''
            - Buy at ask price
            '''
            amount_buy = 100
            inventory += amount_buy
            inventory_history.append(inventory_history[len(inventory_history)-1] + amount_buy)
            buy_bool = True
            
        # sell
        elif pred < (current_bid - buffer):
            '''
            Sell at bid price
            '''
            amount_sell = 100
            inventory -= amount_sell
            inventory_history.append(inventory_history[len(inventory_history)-1] - amount_sell)
            buy_bool = False
        else:
            continue
            
        
        # liquidate
        #if i % 60 == 0 and i > 0:
        if buy_bool == True:
            
            ask_next = data_orig.loc[i+1, 'direct.ask1']
            pnl_lstm += (ask_next - current_ask) * amount_buy
            pnl_history_by_trade.append((ask_next - current_ask) * amount_buy)
            
        else:
            bid_next = data_orig.loc[i+1, 'direct.bid1']
            pnl_lstm += (bid_next - current_bid) * amount_sell
            pnl_history_by_trade.append((bid_next - current_bid) * amount_sell)
        
        pnl_history_lstm.append(pnl_lstm) 
    

In [ ]:
import matplotlib.pyplot as plt

plt.plot(pnl_history_lstm)
plt.show()

In [ ]:
inventory_history

# PnL (Classification)

In [ ]:
starting_money = 100000
stop_loss = -50000
pnl_lstm = 0
pnl_history_lstm = []
pnl_history_by_trade = []
prob_limit = 0.8
inventory = [1000]

test_data = data_orig.loc[train_size:len(data_orig),:].reset_index(drop = True)

if response_type == 'Classification':
    for i in range(len(testX)):
        pred = preds[i]
        prob = preds_probs[i][pred]
        if prob < prob_limit:
            continue

        '''
        At time t:
         - if pred > current:
             - buy at ask price
         
         - if pred < current:
             - sell at bid price
             
             
        At time t+1:
        Liquidate assets
        
         - if bought at time t:
             - sell at bid price at t + 1
             
         - if sold at time t:
             - buy at bid price at t + 1
             
        '''
        #buy_bool = None
        #buy_price = None
        #sell_price = None
        #amount_buy = 1
        #amount_sell = 1
        
        # buy
        if pred == 1:
            '''
            - Buy at ask price
            - 
            '''
            buy_price = test_data.loc[i, 'direct.ask1']
            amount_buy = 100*prob
            buy_bool = True
            
        
        # sell
        elif pred == 2:
            '''
            Sell at bid price
            '''
            #if inventory - 100*prob >= 0:
            sell_price = test_data.loc[i, 'direct.bid1']
            amount_sell = 100*prob
            buy_bool = False
        else:
            continue
            
        
        if buy_bool == True:
            ask_next = test_data.loc[i+1, 'direct.ask1']
            pnl_lstm += (ask_next - buy_price) * amount_buy
            pnl_history_by_trade.append((ask_next - buy_price) * amount_buy)

        else:
            bid_next = test_data.loc[i+1, 'direct.bid1']
            pnl_lstm += (bid_next - sell_price) * amount_sell
            pnl_history_by_trade.append((bid_next - sell_price) * amount_sell)

        
        
        pnl_history_lstm.append(pnl_lstm) 
    

In [ ]:
import matplotlib.pyplot as plt

plt.plot(pnl_history_lstm)
plt.show()

In [ ]:
len([x for x in pnl_history_by_trade if x > 0])

In [ ]:
len([x for x in pnl_history_by_trade if x < 0])

Naive strategy: predict mid price/vwap/AREA for the next time period. If we predict the this metric will go up, we buy. If we predict the metric will go down, we sell. We close positions after each time period. Calculate PnL for that period as ____

# Ignore

In [ ]:
# Class weights to change
class_weight = {0 : 1.,
    1: 15.,
    2: 18.} 

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(10, input_shape=(5,69), return_sequences=False))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.fit(trainX, trainY, 
          epochs=2,  
          batch_size=10, 
          #verbose=2, 
          class_weight = class_weight,
          callbacks=[TensorBoard(log_dir='Logs/testlog', write_graph=True)])

In [ ]:
preds = model.predict(testX).argmax(axis=-1)
pd.Series(preds).value_counts()

In [ ]:
precision_recall_fscore_support(np.argmax(testY, axis=1), preds)